In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam

In [2]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.l1 = nn.Linear(3, 3)
    self.l2 = nn.Linear(3, 3)
  
  def forward(self, x):
    xa = self.l1(x)
    xb = F.relu(xa)
    xc = self.l2(xb)
    return xc
  

In [3]:
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.l1 = nn.Linear(6, 3)
  
  def forward(self, x1, x2):
    x = torch.cat((x1, x2), dim=1)
    return self.l1(x)


In [4]:
encoder = Encoder()
decoder = Decoder()
optimizer_encoder = Adam(encoder.parameters(), lr=0.001)
optimizer_decoder = Adam(decoder.parameters(), lr=0.001)
criterion = nn.MSELoss()


In [6]:
epochs = 10
examples_per_enc = 100
batch_size = 32
hidden_dim = 3

for i in range(epochs):
  x1 = torch.randn(batch_size, hidden_dim)
  enc = encoder(x1)
  for j in range(examples_per_enc):
    x2 = torch.randn(batch_size, hidden_dim)
    y = torch.randn(batch_size, hidden_dim)
    y_hat = decoder(enc, x2)
    loss = criterion(y_hat, y)
    loss.backward()
    optimizer_decoder.step()
    optimizer_decoder.zero_grad()
  optimizer_encoder.step()
  optimizer_encoder.zero_grad()


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

# Notes
Two ideas:
- Do a separate loss_sum.backward() for encoder
  - retain_graph=False
- Use retain_graph=True
  - Check with torchviz
  - Control the computation graph (should not be growing)
  - Check all reassignments of variables